In [1]:
import polars as pl
from datetime import datetime
import unicodedata
import os

In [2]:
df = pl.read_csv('afluenciastc_desglosado_12_2024.csv')
df #854,685

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
str,str,i64,str,str,str,i64
"""2021-01-01""","""Enero""",2021,"""Línea 1""","""Balbuena""","""Boleto""",1848
"""2021-01-01""","""Enero""",2021,"""Línea 1""","""Balbuena""","""Prepago""",790
"""2021-01-01""","""Enero""",2021,"""Línea 1""","""Balbuena""","""Gratuidad""",325
"""2021-01-01""","""Enero""",2021,"""Línea 1""","""Balderas""","""Boleto""",1103
"""2021-01-01""","""Enero""",2021,"""Línea 1""","""Balderas""","""Prepago""",922
…,…,…,…,…,…,…
"""2024-12-31""","""Diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Prepago""",14662
"""2024-12-31""","""Diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Gratuidad""",1537
"""2024-12-31""","""Diciembre""",2024,"""Linea 12""","""Mixcoac""","""Boleto""",0


In [3]:
# Convertir a fechas 
df = df.with_columns(pl.col('fecha').cast(pl.Date))
df = df.sort('fecha')
df.head()

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2021-01-01,"""Enero""",2021,"""Línea 1""","""Balbuena""","""Boleto""",1848
2021-01-01,"""Enero""",2021,"""Línea 1""","""Balbuena""","""Prepago""",790
2021-01-01,"""Enero""",2021,"""Línea 1""","""Balbuena""","""Gratuidad""",325
2021-01-01,"""Enero""",2021,"""Línea 1""","""Balderas""","""Boleto""",1103
2021-01-01,"""Enero""",2021,"""Línea 1""","""Balderas""","""Prepago""",922


In [4]:
df.schema

Schema([('fecha', Date),
        ('mes', String),
        ('anio', Int64),
        ('linea', String),
        ('estacion', String),
        ('tipo_pago', String),
        ('afluencia', Int64)])

In [5]:
# Revisando columnas

columna = df.select(pl.col("anio").unique())
columna
#Fecha - Date
#Mes - Tiene un valores en NA
#Anio - solo cubre 4 años en el DataSet desplegado
#Lineas - tienen nombres con acento y sin acento, por lo que son más
#Estaciones - Son 195 oficialmente pero marca 216
#Tipo pago - Solo son 3
#Afluencia - Int64


anio
i64
2021
2022
2023
2024


In [6]:
df.filter(pl.col('mes')=='NA')

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Boleto""",0
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Prepago""",22826
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Gratuidad""",3930
2023-12-01,"""NA""",2023,"""Línea 1""","""Zaragoza""","""Boleto""",0
2023-12-01,"""NA""",2023,"""Línea 1""","""Zaragoza""","""Prepago""",15315
…,…,…,…,…,…,…
2023-12-31,"""NA""",2023,"""Línea 12""","""Insurgentes Sur""","""Prepago""",6129
2023-12-31,"""NA""",2023,"""Línea 12""","""Insurgentes Sur""","""Gratuidad""",897
2023-12-31,"""NA""",2023,"""Línea 12""","""Mixcoac""","""Boleto""",0


# Limpiar

## Meses

In [7]:
#Nulos
df.select(pl.col('mes').is_null()).sum()
#No hay nulos

mes
u32
0


In [8]:
df.filter(pl.col('mes')=='NA')#Hay 18k valores NA pero tienen correspondencia con fecha (fecha no tiene nulos)

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Boleto""",0
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Prepago""",22826
2023-12-01,"""NA""",2023,"""Línea 1""","""Pantitlán""","""Gratuidad""",3930
2023-12-01,"""NA""",2023,"""Línea 1""","""Zaragoza""","""Boleto""",0
2023-12-01,"""NA""",2023,"""Línea 1""","""Zaragoza""","""Prepago""",15315
…,…,…,…,…,…,…
2023-12-31,"""NA""",2023,"""Línea 12""","""Insurgentes Sur""","""Prepago""",6129
2023-12-31,"""NA""",2023,"""Línea 12""","""Insurgentes Sur""","""Gratuidad""",897
2023-12-31,"""NA""",2023,"""Línea 12""","""Mixcoac""","""Boleto""",0


### Pasos a seguir
- Minusculas meses
- Extraer de fecha el mes y asociarlo a columna mes

In [9]:
df = df.with_columns(pl.col('mes').str.to_lowercase())
df

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Boleto""",1848
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Prepago""",790
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Gratuidad""",325
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Boleto""",1103
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Prepago""",922
…,…,…,…,…,…,…
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Prepago""",14662
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Gratuidad""",1537
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Mixcoac""","""Boleto""",0


In [10]:
meses = {
    'january'  :'enero',
    'february' :'febrero',
    'march'    :'marzo',
    'april'    :'abril',
    'may'      :'mayo',
    'june'     :'junio',
    'july'     :'julio',
    'august'   :'agosto',
    'september':'septiembre',
    'october'  :'octubre',
    'november' :'noviembre',
    'december' :'diciembre'
}

months = pl.col('fecha').dt.strftime("%B").str.to_lowercase().alias("month")
meses_final = months.replace(meses)

In [11]:
meses_final

<Expr ['col("fecha").dt.to_string().st…'] at 0x1A9FABD1400>

In [12]:
df = df.with_columns(
    pl.when(pl.col("mes")=='NA')  # Si el valor en "meses" es NA
    .then(meses_final)                # Asignar el mes traducido
    .otherwise(pl.col("mes"))         # De lo contrario, mantener el valor original
    .alias("meses")                     # Actualizar la columna "meses"
)
df

fecha,mes,anio,linea,estacion,tipo_pago,afluencia,meses
date,str,i64,str,str,str,i64,str
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Boleto""",1848,"""enero"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Prepago""",790,"""enero"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Gratuidad""",325,"""enero"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Boleto""",1103,"""enero"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Prepago""",922,"""enero"""
…,…,…,…,…,…,…,…
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Prepago""",14662,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Insurgentes Sur""","""Gratuidad""",1537,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea 12""","""Mixcoac""","""Boleto""",0,"""diciembre"""


In [13]:
df.filter(pl.col('mes')=='NA')#Hay 18k valores NA pero tienen correspondencia con fecha (fecha no tiene nulos)

fecha,mes,anio,linea,estacion,tipo_pago,afluencia,meses
date,str,i64,str,str,str,i64,str


### Lineas metro

In [14]:
lineas = df['linea'].unique().to_list()
lineas #Son 24 lineas significando que podrían estar duplicadas o hay linea con acento y sin acento

['Linea 3',
 'Linea 7',
 'Línea 1',
 'Línea 6',
 'Linea B',
 'Línea 12',
 'Linea 4',
 'Linea 8',
 'Linea 9',
 'Línea B',
 'Linea 2',
 'Línea A',
 'Linea A',
 'Línea 2',
 'Linea 12',
 'Linea 5',
 'Línea 3',
 'Línea 7',
 'Línea 5',
 'Linea 1',
 'Línea 4',
 'Linea 6',
 'Línea 8',
 'Línea 9']

In [15]:
# Comparando lineas
df.select(pl.col('linea')=='Linea A').sum(),df.select(pl.col('linea')=='Línea A').sum() #Hay mas Líneas que Lineas

(shape: (1, 1)
 ┌───────┐
 │ linea │
 │ ---   │
 │ u32   │
 ╞═══════╡
 │ 9180  │
 └───────┘,
 shape: (1, 1)
 ┌───────┐
 │ linea │
 │ ---   │
 │ u32   │
 ╞═══════╡
 │ 34650 │
 └───────┘)

In [16]:
df.filter(
    (pl.col('linea')=='Linea A') & (pl.col('fecha')==datetime(2024,12,31))
)

fecha,mes,anio,linea,estacion,tipo_pago,afluencia,meses
date,str,i64,str,str,str,i64,str
2024-12-31,"""diciembre""",2024,"""Linea A""","""PantitlÃ¡n""","""Boleto""",0,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""PantitlÃ¡n""","""Prepago""",19172,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""PantitlÃ¡n""","""Gratuidad""",1881,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""AgrÃ­cola Oriental""","""Boleto""",0,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""AgrÃ­cola Oriental""","""Prepago""",7319,"""diciembre"""
…,…,…,…,…,…,…,…
2024-12-31,"""diciembre""",2024,"""Linea A""","""Los Reyes""","""Prepago""",13934,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""Los Reyes""","""Gratuidad""",2138,"""diciembre"""
2024-12-31,"""diciembre""",2024,"""Linea A""","""La Paz""","""Boleto""",0,"""diciembre"""


In [17]:
df.filter(
    (pl.col('linea')=='Línea A') & (pl.col('fecha')==datetime(2024,12,31))
)

fecha,mes,anio,linea,estacion,tipo_pago,afluencia,meses
date,str,i64,str,str,str,i64,str


Se analizara si la diferencia que tienen los dos conjuntos es debido a la fecha o al contenido en alguna columna.
- Primero obteniendo el rango de fechas que contiene cada conjunto
- Despues con algún tipo de Join para saber si tienen cosas en común
Aunque con la fecha debería de bastar

In [18]:
df.filter(pl.col('linea')=='Linea 12')['anio'].unique(),df.filter(pl.col('linea')=='Línea 12')['anio'].unique()
# 2024 es la única fecha en común entre las líneas

(shape: (1,)
 Series: 'anio' [i64]
 [
 	2024
 ],
 shape: (4,)
 Series: 'anio' [i64]
 [
 	2021
 	2022
 	2023
 	2024
 ])

In [19]:
#Buscando diferencias o similitudes entre 2024 y por linea
#Primero hay que normalizar el texto
def elimina_acentos(texto):
    if texto is None:
        return None
    texto_normalizado = unicodedata.normalize('NFD', texto)
    texto_sin_acentos = ''.join(c for c in texto_normalizado if not unicodedata.combining(c))
    return texto_sin_acentos

In [20]:
df = df.with_columns(pl.col("linea").map_elements(elimina_acentos, return_dtype=pl.String).alias("texto_sin_acentos"))
df.head()

fecha,mes,anio,linea,estacion,tipo_pago,afluencia,meses,texto_sin_acentos
date,str,i64,str,str,str,i64,str,str
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Boleto""",1848,"""enero""","""Linea 1"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Prepago""",790,"""enero""","""Linea 1"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balbuena""","""Gratuidad""",325,"""enero""","""Linea 1"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Boleto""",1103,"""enero""","""Linea 1"""
2021-01-01,"""enero""",2021,"""Línea 1""","""Balderas""","""Prepago""",922,"""enero""","""Linea 1"""


In [21]:
df = df.drop('linea')
df = df.select(['fecha','meses','anio','texto_sin_acentos','estacion','tipo_pago','afluencia'])
df = df.rename({'meses':'mes','texto_sin_acentos':'linea'})
df.head()

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2021-01-01,"""enero""",2021,"""Linea 1""","""Balbuena""","""Boleto""",1848
2021-01-01,"""enero""",2021,"""Linea 1""","""Balbuena""","""Prepago""",790
2021-01-01,"""enero""",2021,"""Linea 1""","""Balbuena""","""Gratuidad""",325
2021-01-01,"""enero""",2021,"""Linea 1""","""Balderas""","""Boleto""",1103
2021-01-01,"""enero""",2021,"""Linea 1""","""Balderas""","""Prepago""",922


Para Línea y Linea deberian haber algunos criterios para saber si no son duplicados:
- El primero es no cambiar los acentos en primera instancia.
- Para cada dia todos los datos pueden repetirse menos en tipo de pago para una estacion en particular
- El maximo por mes de una linea (sumando los dos conjuntos) es total de dias del mes x numero de estaciones x tipo_pago

## Linea A

In [22]:
fecha_inicio = datetime(2024,1,1)
fecha_fin    = datetime(2024,12,31)

df_linea_A = df.filter(pl.col('linea')=="Linea A")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [23]:
df_linea_A = df_linea_A.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_A = df_linea_A.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_A = df_linea_A.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_A = df_linea_A.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [24]:
estaciones = df_linea_A.select(pl.col("estacion").unique()).to_series().to_list()

In [25]:
estaciones

['agra\xadcola oriental',
 'canal de san juan',
 'santa marta',
 'acatitla',
 'penon viejo',
 'guelatao',
 'pantitlan',
 'pea±a³n viejo',
 'tepalcates',
 'pantitla¡n',
 'los reyes',
 'la paz',
 'agricola oriental']

In [26]:
df_linea_A.filter(pl.col("estacion")=="pantitla¡n")

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2024-12-01,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""boleto""",0
2024-12-01,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""prepago""",14719
2024-12-01,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""gratuidad""",1435
2024-12-02,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""boleto""",0
2024-12-02,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""prepago""",32983
…,…,…,…,…,…,…
2024-12-30,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""prepago""",25495
2024-12-30,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""gratuidad""",4027
2024-12-31,"""diciembre""",2024,"""Linea A""","""pantitla¡n""","""boleto""",0


In [27]:
df_linea_A = df_linea_A.with_columns(pl.col('estacion').str.replace("penon viejo","peñon viejo"))

In [28]:
df_linea_A.head()

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2021-01-01,"""enero""",2021,"""Linea A""","""acatitla""","""boleto""",1814
2021-01-01,"""enero""",2021,"""Linea A""","""acatitla""","""prepago""",2265
2021-01-01,"""enero""",2021,"""Linea A""","""acatitla""","""gratuidad""",1135
2021-01-01,"""enero""",2021,"""Linea A""","""agricola oriental""","""boleto""",3610
2021-01-01,"""enero""",2021,"""Linea A""","""agricola oriental""","""prepago""",804


In [29]:
df2 = df_linea_A.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""peñon viejo""","""prepago""",1430
"""pantitlan""","""prepago""",1430
"""la paz""","""gratuidad""",1461
"""santa marta""","""gratuidad""",1461
"""peñon viejo""","""boleto""",1430
"""tepalcates""","""prepago""",1461
"""canal de san juan""","""boleto""",1461
"""los reyes""","""prepago""",1461
"""agra­cola oriental""","""gratuidad""",31


In [30]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,10227,43830
"""prepago""",1430,4290,43830
"""boleto""",1430,4290,43830
"""boleto""",31,93,43830
"""boleto""",1461,10227,43830
"""gratuidad""",31,93,43830
"""prepago""",31,93,43830
"""gratuidad""",1430,4290,43830
"""gratuidad""",1461,10227,43830


In [31]:
#df_linea_A.write_csv("./metro_cdmx/linea_A_limpio.csv")

## Linea 1

In [32]:
fecha_inicio = datetime(2024,1,1)
fecha_fin    = datetime(2024,12,31)

df_linea_1 = df.filter(pl.col('linea')=="Linea 1")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [33]:
df_linea_1 = df_linea_1.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_1 = df_linea_1.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_1 = df_linea_1.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_1 = df_linea_1.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [34]:
estaciones = df_linea_1.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

28

In [35]:
df_linea_1.filter(pl.col("estacion")=="juanacatla¡n")

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2024-12-01,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""boleto""",0
2024-12-01,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""prepago""",0
2024-12-01,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""gratuidad""",0
2024-12-02,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""boleto""",0
2024-12-02,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""prepago""",0
…,…,…,…,…,…,…
2024-12-30,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""prepago""",0
2024-12-30,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""gratuidad""",0
2024-12-31,"""diciembre""",2024,"""Linea 1""","""juanacatla¡n""","""boleto""",0


In [36]:
df_linea_1 = df_linea_1.with_columns(pl.col('estacion').str.replace("juanacatla¡n","juanacatlan"))

In [37]:
df_linea_1.head()

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
date,str,i64,str,str,str,i64
2021-01-01,"""enero""",2021,"""Linea 1""","""balbuena""","""boleto""",1848
2021-01-01,"""enero""",2021,"""Linea 1""","""balbuena""","""prepago""",790
2021-01-01,"""enero""",2021,"""Linea 1""","""balbuena""","""gratuidad""",325
2021-01-01,"""enero""",2021,"""Linea 1""","""balderas""","""boleto""",1103
2021-01-01,"""enero""",2021,"""Linea 1""","""balderas""","""prepago""",922


In [38]:
df2 = df_linea_1.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""cuauhtemoc""","""boleto""",1430
"""pino sua¡rez""","""boleto""",31
"""observatorio""","""boleto""",1461
"""san la¡zaro""","""boleto""",31
"""ga³mez farias""","""prepago""",31
"""boulevard puerto aa©reo""","""prepago""",31
"""merced""","""gratuidad""",1461
"""tacubaya""","""gratuidad""",1461
"""pino sua¡rez""","""prepago""",31


In [39]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""boleto""",1461,18993,87660
"""gratuidad""",1461,18993,87660
"""boleto""",31,217,87660
"""gratuidad""",31,217,87660
"""prepago""",31,217,87660
"""prepago""",1430,10010,87660
"""gratuidad""",1430,10010,87660
"""prepago""",1461,18993,87660
"""boleto""",1430,10010,87660


In [40]:
#df_linea_1.write_csv('metro_cdmx/linea_1_limpio.csv')

## Linea 2

In [41]:
df_linea_2 = df.filter(pl.col('linea')=="Linea 2")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [42]:
df_linea_2 = df_linea_2.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_2 = df_linea_2.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_2 = df_linea_2.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_2 = df_linea_2.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [43]:
estaciones = df_linea_2.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

29

In [44]:
df_linea_2 = df_linea_2.with_columns(pl.col('estacion').str.replace("pino sua¡rez","pino suarez"))

In [45]:
df2 = df_linea_2.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""nativitas""","""boleto""",1461
"""portales""","""boleto""",1461
"""san antonio abad""","""boleto""",1461
"""san cosme""","""gratuidad""",1461
"""xola""","""gratuidad""",1461
"""cuitla¡huac""","""boleto""",31
"""chabacano""","""boleto""",1461
"""tasquea±a""","""gratuidad""",31
"""tasquena""","""prepago""",1430


In [46]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""boleto""",1430,5720,105192
"""boleto""",1461,29220,105192
"""prepago""",1461,29220,105192
"""gratuidad""",31,124,105192
"""gratuidad""",1461,29220,105192
"""prepago""",1430,5720,105192
"""prepago""",31,124,105192
"""gratuidad""",1430,5720,105192
"""boleto""",31,124,105192


In [47]:
#df_linea_2.write_csv('metro_cdmx/linea_2_limpio.csv')

## Linea 3

In [48]:
df_linea_3 = df.filter(pl.col('linea')=="Linea 3")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [49]:
df_linea_3 = df_linea_3.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_3 = df_linea_3.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_3 = df_linea_3.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_3 = df_linea_3.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [50]:
estaciones = df_linea_3.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

28

In [51]:
df_linea_3 = df_linea_3.with_columns(pl.col('estacion').str.replace("jua¡rez","juarez"))

In [52]:
df2 = df_linea_3.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""hidalgo""","""boleto""",1461
"""coyoaca¡n""","""boleto""",31
"""copilco""","""boleto""",1461
"""zapata""","""prepago""",1461
"""nia±os ha©roes""","""boleto""",31
"""universidad""","""gratuidad""",1461
"""tlatelolco""","""gratuidad""",1461
"""miguel angel de quevedo""","""gratuidad""",1430
"""hospital general""","""prepago""",1461


In [53]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1430,8580,92043
"""boleto""",1430,8580,92043
"""prepago""",1461,21915,92043
"""prepago""",31,186,92043
"""gratuidad""",31,186,92043
"""boleto""",1461,21915,92043
"""gratuidad""",1430,8580,92043
"""boleto""",31,186,92043
"""gratuidad""",1461,21915,92043


In [54]:
#df_linea_3.write_csv('metro_cdmx/linea_3_limpio.csv')

## Linea 4

In [55]:
df_linea_4 = df.filter(pl.col('linea')=="Linea 4")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [56]:
df_linea_4 = df_linea_4.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_4 = df_linea_4.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_4 = df_linea_4.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_4 = df_linea_4.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [57]:
estaciones = df_linea_4.select(pl.col("estacion").unique()).to_series().to_list()
estaciones #Hay ocho estaciones de mas

['martin carrera',
 'talisman',
 'marta\xadn carrera',
 'canal del norte',
 'candelaria',
 'jamaica',
 'talisma¡n',
 'consulado',
 'bondojito',
 'santa anita',
 'fray servando',
 'morelos']

In [58]:
df_linea_4 = df_linea_4.with_columns(pl.col('estacion').str.replace("talisma¡n","talisman"))

In [59]:
df2 = df_linea_4.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""fray servando""","""prepago""",1461
"""consulado""","""prepago""",1461
"""talisman""","""prepago""",1461
"""santa anita""","""boleto""",1461
"""talisman""","""gratuidad""",1461
"""santa anita""","""prepago""",1461
"""candelaria""","""gratuidad""",1461
"""talisman""","""boleto""",1461
"""bondojito""","""prepago""",1461


In [60]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""gratuidad""",1430,1430,43830
"""prepago""",1461,13149,43830
"""prepago""",31,31,43830
"""boleto""",1461,13149,43830
"""prepago""",1430,1430,43830
"""boleto""",1430,1430,43830
"""gratuidad""",31,31,43830
"""gratuidad""",1461,13149,43830
"""boleto""",31,31,43830


In [61]:
#df_linea_4.write_csv('metro_cdmx/linea_4_limpio.csv')

## Linea 5

In [67]:
df_linea_5 = df.filter(pl.col('linea')=="Linea 5")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [68]:
df_linea_5 = df_linea_5.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_5 = df_linea_5.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_5 = df_linea_5.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_5 = df_linea_5.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [83]:
estaciones = df_linea_5.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

13

In [81]:
df_linea_5 = df_linea_5.with_columns(pl.col('estacion').str.replace("polita©cnico","politecnico"))

In [84]:
df2 = df_linea_5.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""hangares""","""boleto""",1461
"""pantitlan""","""gratuidad""",1461
"""politecnico""","""gratuidad""",1461
"""valle gomez""","""prepago""",1461
"""oceania""","""prepago""",1461
"""politecnico""","""prepago""",1461
"""terminal aerea""","""gratuidad""",1461
"""terminal aerea""","""prepago""",1461
"""politecnico""","""boleto""",1461


In [85]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""gratuidad""",1461,18993,56979
"""boleto""",1461,18993,56979
"""prepago""",1461,18993,56979


In [86]:
df_linea_5.write_csv('metro_cdmx/linea_5_limpio.csv')

## Linea 6

In [87]:
df_linea_6 = df.filter(pl.col('linea')=="Linea 6")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [88]:
df_linea_6 = df_linea_6.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_6 = df_linea_6.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_6 = df_linea_6.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_6 = df_linea_6.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [101]:
estaciones = df_linea_6.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

11

In [99]:
df_linea_6 = df_linea_6.with_columns(pl.col('estacion').str.replace("instituto del petra³leo","instituto del petroleo"))

In [102]:
df2 = df_linea_6.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""el rosario""","""prepago""",1461
"""la villa/basilica""","""gratuidad""",1461
"""deportivo 18 de marzo""","""boleto""",1461
"""lindavista""","""gratuidad""",1461
"""tezozomoc""","""boleto""",1461
"""la villa/basilica""","""boleto""",1461
"""la villa/basilica""","""prepago""",1461
"""ferreria/arena ciudad de mexic…","""boleto""",1461
"""tezozomoc""","""gratuidad""",1461


In [103]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,16071,48213
"""gratuidad""",1461,16071,48213
"""boleto""",1461,16071,48213


In [105]:
df_linea_6.write_csv('metro_cdmx/linea_6_limpio.csv')

## Linea 7

In [133]:
df_linea_7 = df.filter(pl.col('linea')=="Linea 7")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [134]:
df_linea_7 = df_linea_7.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_7 = df_linea_7.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_7 = df_linea_7.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_7 = df_linea_7.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [140]:
estaciones = df_linea_7.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

14

In [138]:
df_linea_7 = df_linea_7.with_columns(pl.col('estacion').str.replace("san joaqua\xadn","san joaquin"))

In [141]:
df2 = df_linea_7.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""mixcoac""","""prepago""",1461
"""tacubaya""","""prepago""",1461
"""auditorio""","""boleto""",1461
"""aquiles serdan""","""gratuidad""",1461
"""camarones""","""prepago""",1461
"""polanco""","""gratuidad""",1461
"""el rosario""","""boleto""",1461
"""san antonio""","""gratuidad""",1461
"""san pedro de los pinos""","""gratuidad""",1461


In [142]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,20454,61362
"""boleto""",1461,20454,61362
"""gratuidad""",1461,20454,61362


In [143]:
df_linea_7.write_csv('metro_cdmx/linea_7_limpio.csv')

## Linea 8

In [144]:
df_linea_8 = df.filter(pl.col('linea')=="Linea 8")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [145]:
df_linea_8 = df_linea_8.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_8 = df_linea_8.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_8 = df_linea_8.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_8 = df_linea_8.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [153]:
estaciones = df_linea_8.select(pl.col("estacion").unique()).to_series().to_list()
estaciones #Hay ocho estaciones de mas

['bellas artes',
 'cerro de la estrella',
 'atlalilco',
 'iztacalco',
 'constitucion de 1917',
 'escuadron 201',
 'santa anita',
 'chabacano',
 'uam-i',
 'obrera',
 'garibaldi/lagunilla',
 'salto del agua',
 'iztapalapa',
 'la viga',
 'aculco',
 'apatlaco',
 'coyuya',
 'doctores',
 'san juan de letran']

In [154]:
df_linea_8 = df_linea_8.with_columns(pl.col('estacion').str.replace("escuadra³n 201","escuadron 201"))

In [155]:
df2 = df_linea_8.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""bellas artes""","""gratuidad""",1461
"""doctores""","""prepago""",1461
"""escuadron 201""","""prepago""",1461
"""aculco""","""boleto""",1461
"""atlalilco""","""boleto""",1461
"""garibaldi/lagunilla""","""prepago""",1461
"""escuadron 201""","""boleto""",1461
"""chabacano""","""gratuidad""",1461
"""apatlaco""","""prepago""",1461


In [156]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,27759,83277
"""gratuidad""",1461,27759,83277
"""boleto""",1461,27759,83277


In [157]:
df_linea_8.write_csv('metro_cdmx/linea_8_limpio.csv')

## Linea 9

In [158]:
df_linea_9 = df.filter(pl.col('linea')=="Linea 9")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [159]:
df_linea_9 = df_linea_9.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_9 = df_linea_9.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_9 = df_linea_9.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_9 = df_linea_9.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [169]:
estaciones = df_linea_9.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

12

In [167]:
df_linea_9 = df_linea_9.with_columns(pl.col('estacion').str.replace("centro ma©dico","centro medico"))

In [170]:
df2 = df_linea_9.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""pantitlan""","""gratuidad""",1461
"""chabacano""","""prepago""",1461
"""tacubaya""","""gratuidad""",1461
"""jamaica""","""gratuidad""",1461
"""centro medico""","""prepago""",1461
"""lazaro cardenas""","""prepago""",1461
"""chilpancingo""","""prepago""",1461
"""velodromo""","""boleto""",1461
"""patriotismo""","""gratuidad""",1461


In [171]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,17532,52596
"""gratuidad""",1461,17532,52596
"""boleto""",1461,17532,52596


In [172]:
df_linea_9.write_csv('metro_cdmx/linea_9_limpio.csv')

## Linea B

In [173]:
df_linea_B = df.filter(pl.col('linea')=="Linea B")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [174]:
df_linea_B = df_linea_B.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_B = df_linea_B.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_B = df_linea_B.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_B = df_linea_B.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [195]:
estaciones = df_linea_B.select(pl.col("estacion").unique()).to_series().to_list()
estaciones #Hay ocho estaciones de mas

['ricardo flores magon',
 'muzquiz',
 'nezahuacoyotl',
 'buenavista',
 'guerrero',
 'impulsora',
 'ciudad azteca',
 'plaza aragon',
 'garibaldi/lagunilla',
 'rio de los remedios',
 'deportivo oceania',
 'romero rubio',
 'oceania',
 'tepito',
 'ecatepec',
 'olimpica',
 'nezahualcoyotl',
 'villa de aragon',
 'bosque de aragon',
 'lagunilla',
 'san lazaro',
 'morelos']

In [194]:
df_linea_B = df_linea_B.with_columns(pl.col('estacion').str.replace("maºzquiz","muzquiz"))

In [196]:
df2 = df_linea_B.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""olimpica""","""gratuidad""",1461
"""guerrero""","""gratuidad""",1461
"""buenavista""","""gratuidad""",1461
"""buenavista""","""prepago""",1461
"""ricardo flores magon""","""boleto""",1461
"""nezahuacoyotl""","""gratuidad""",31
"""oceania""","""boleto""",1461
"""san lazaro""","""prepago""",1461
"""tepito""","""gratuidad""",1461


In [197]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""prepago""",1461,29220,92043
"""prepago""",1430,1430,92043
"""boleto""",31,31,92043
"""boleto""",1461,29220,92043
"""gratuidad""",1461,29220,92043
"""gratuidad""",1430,1430,92043
"""prepago""",31,31,92043
"""boleto""",1430,1430,92043
"""gratuidad""",31,31,92043


In [198]:
df_linea_B.write_csv('metro_cdmx/linea_B_limpio.csv')

## Linea 12

In [199]:
df_linea_12 = df.filter(pl.col('linea')=="Linea 12")
#df_linea_A = df.filter((pl.col('linea')=="Linea A")&(pl.col('fecha').is_between(fecha_inicio,fecha_fin)))

In [200]:
df_linea_12 = df_linea_12.with_columns(pl.col("tipo_pago").map_elements(elimina_acentos, return_dtype=pl.String).alias("tipo_pago"))
df_linea_12 = df_linea_12.with_columns(pl.col("tipo_pago").str.to_lowercase().alias("tipo_pago"))
df_linea_12 = df_linea_12.with_columns(pl.col("estacion").map_elements(elimina_acentos, return_dtype=pl.String).alias("estacion"))
df_linea_12 = df_linea_12.with_columns(pl.col("estacion").str.to_lowercase().alias("estacion"))

In [212]:
estaciones = df_linea_12.select(pl.col("estacion").unique()).to_series().to_list()
len(estaciones) #Hay ocho estaciones de mas

20

In [210]:
df_linea_12 = df_linea_12.with_columns(pl.col('estacion').str.replace("san andra©s tomatla¡n","san andres tomatlan"))

In [213]:
df2 = df_linea_12.group_by(['estacion','tipo_pago']).agg(
    pl.len().alias("pago_estacion")
)
df2.head(10)

estacion,tipo_pago,pago_estacion
str,str,u32
"""mexicaltzingo""","""prepago""",1461
"""mixcoac""","""prepago""",1461
"""nopalera""","""gratuidad""",1461
"""eje central""","""gratuidad""",1461
"""san andres tomatlan""","""boleto""",1461
"""nopalera""","""prepago""",1461
"""parque de los venados""","""prepago""",1461
"""ermita""","""boleto""",1461
"""tlahuac""","""prepago""",1461


In [214]:
df2.group_by(["tipo_pago","pago_estacion"]).agg(
    pl.sum("pago_estacion").alias("suma_pago")
).with_columns(pl.col("suma_pago").sum().alias("total_anual"))

tipo_pago,pago_estacion,suma_pago,total_anual
str,u32,u32,u32
"""boleto""",1461,29220,87660
"""prepago""",1461,29220,87660
"""gratuidad""",1461,29220,87660


In [215]:
df_linea_12.write_csv('metro_cdmx/linea_12_limpio.csv')

# DF GENERAL

In [17]:
path = 'metro_cdmx/'
content = os.listdir(path)
content

['linea_12_limpio.csv',
 'linea_1_limpio.csv',
 'linea_2_limpio.csv',
 'linea_3_limpio.csv',
 'linea_4_limpio.csv',
 'linea_5_limpio.csv',
 'linea_6_limpio.csv',
 'linea_7_limpio.csv',
 'linea_8_limpio.csv',
 'linea_9_limpio.csv',
 'linea_A_limpio.csv',
 'linea_B_limpio.csv']

In [18]:
df_lista = [pl.read_csv('metro_cdmx/'+ i,encoding='latin1') for i in content]
df_lista

[shape: (87_660, 7)
 ┌────────────┬───────────┬──────┬──────────┬──────────────────────────┬───────────┬───────────┐
 │ fecha      ┆ mes       ┆ anio ┆ linea    ┆ estacion                 ┆ tipo_pago ┆ afluencia │
 │ ---        ┆ ---       ┆ ---  ┆ ---      ┆ ---                      ┆ ---       ┆ ---       │
 │ str        ┆ str       ┆ i64  ┆ str      ┆ str                      ┆ str       ┆ i64       │
 ╞════════════╪═══════════╪══════╪══════════╪══════════════════════════╪═══════════╪═══════════╡
 │ 2021-01-01 ┆ enero     ┆ 2021 ┆ Linea 12 ┆ hospital 20 de noviembre ┆ boleto    ┆ 0         │
 │ 2021-01-01 ┆ enero     ┆ 2021 ┆ Linea 12 ┆ hospital 20 de noviembre ┆ prepago   ┆ 832       │
 │ 2021-01-01 ┆ enero     ┆ 2021 ┆ Linea 12 ┆ hospital 20 de noviembre ┆ gratuidad ┆ 733       │
 │ 2021-01-01 ┆ enero     ┆ 2021 ┆ Linea 12 ┆ atlalilco                ┆ boleto    ┆ 0         │
 │ 2021-01-01 ┆ enero     ┆ 2021 ┆ Linea 12 ┆ atlalilco                ┆ prepago   ┆ 1141      │
 │ …      

In [19]:
df_general = pl.concat(df_lista)

In [20]:
df_general

fecha,mes,anio,linea,estacion,tipo_pago,afluencia
str,str,i64,str,str,str,i64
"""2021-01-01""","""enero""",2021,"""Linea 12""","""hospital 20 de noviembre""","""boleto""",0
"""2021-01-01""","""enero""",2021,"""Linea 12""","""hospital 20 de noviembre""","""prepago""",832
"""2021-01-01""","""enero""",2021,"""Linea 12""","""hospital 20 de noviembre""","""gratuidad""",733
"""2021-01-01""","""enero""",2021,"""Linea 12""","""atlalilco""","""boleto""",0
"""2021-01-01""","""enero""",2021,"""Linea 12""","""atlalilco""","""prepago""",1141
…,…,…,…,…,…,…
"""2024-12-31""","""diciembre""",2024,"""Linea B""","""plaza aragon""","""prepago""",10348
"""2024-12-31""","""diciembre""",2024,"""Linea B""","""plaza aragon""","""gratuidad""",2232
"""2024-12-31""","""diciembre""",2024,"""Linea B""","""ciudad azteca""","""boleto""",0


In [21]:
df_general.write_csv('metro_cdmx/afluencia_desglosado_procesado.csv')
df_general.write_json('metro_cdmx/afluencia_desglosado_procesado.json')